# Task04. 下游任务的端到端学习

# 1. 端到端学习的重要性

### 在Task04中我们使用PyPOTS对自定义数据集进行了两阶段建模, 即在上游先对数据集中的缺失值进行插补, 然后对处理好的数据集进行下游任务建模. 同时, 我们也强调了缺失值本身也是数据集的特性, 数据的缺失模式可能携带了额外的信息来表征数据采集对象的状态, 而这些信息在两阶段处理中可能会丢失, 因为插补值是模型根据观测到的数据分布推测出来的, 插补后下游算法无法知道原数据中的缺失模式, 也就无法充分利用这部分信息来学习. 端到端学习则是使用一个模型直接接受包含缺失值的数据然后在特定任务上进行学习. 虽然在很多时候我们无法提前判断在某个时序数据集上是使用两阶段方法还是端到端方法能够取得更好的效果 (因为最终的效果涉及到很多方面, 包括但不限于模型自身的能力, 超参调优 等等), 但是端到端方法显然拥有更大的潜力

# 2. 使用BRITS直接在PhysioNet2012上进行分类

### 2.1 数据加载

In [ ]:
from benchpots.datasets import preprocess_physionet2012

physionet2012_dataset = preprocess_physionet2012(
    subset="set-a", 
    pattern="point", 
    rate=0.1,
)

dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
    "y": physionet2012_dataset['train_y'],
}

dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "y": physionet2012_dataset['val_y'],
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "y": physionet2012_dataset['test_y'],
}

### 2.2 使用PyPOTS进行模型训练

In [ ]:
from pypots.classification import BRITS

brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_classes=physionet2012_dataset["n_classes"],
    rnn_hidden_size=128,
    epochs=20,
    patience=5,
)

brits.fit(dataset_for_training, dataset_for_validating)

### 2.3 计算分类精度

In [ ]:
from pypots.nn.functional.classification import calc_binary_classification_metrics

brits_results = brits.predict(dataset_for_testing)
brits_prediction = brits_results["classification"]

classification_metrics=calc_binary_classification_metrics(
    brits_prediction, dataset_for_testing["y"]
)
print(f"BRITS在测试集上的ROC-AUC为: {classification_metrics['roc_auc']:.4f}\n")
print(f"BRITS在测试集上的PR-AUC为: {classification_metrics['pr_auc']:.4f}\n")